## 1. Введение в базы данных (далее БД)

Хранение информации на компьютере - давно привычное явление. Люди хранят пароли в текстовых файлах, документы в формате Word, однако такая организация хранения мало пригодна при большом объеме информации по нескольким причинам:
   - в них много дублирования, из-за чего требуется значительно больше места на жестком диске,
   - а поиск работает медленно и слишком «дорогой» при сколько-либо значимом количестве обращений.
   
Разработчики промышленного программного обеспечения столкнулись с этими проблемами достаточно давно, и в качестве одного из решений еще в 1970 году Эдгар Кодд предложил реляционную модель данных (файловые базы данных появились еще раньше — в 1955 году).
Эта идея развилась в привычные сегодня для почти каждого программиста реляционные базы данных (БД).
База данных — это непосредственное хранилище информации, которое без инструментов для взаимодействия с ним не очень то и полезно. Такой интерфейс для общения с БД разработчикам и системным администраторам предоставляет специальное программное обеспечение — Системы управления базами данных (СУБД).

## 2. Пример подключения к базе данных
Мы будем работать с базами данных и языком SQL с помощью Python.
Так как различных СУБД достаточно много, крайне неудобно было бы, если при переходе на новую СУБД приходилось бы с нуля изучать библиотеку для работы с ней. Чтобы избежать таких ситуаций, есть специальный стандарт PEP 249 (Python Database API Specification v2.0), в котором, помимо всего прочего, описано, какой интерфейс должна предоставлять программисту любая библиотека для работы с базами данных. Поэтому, какую бы СУБД вы не выбрали для управления хранением данных вашего приложения, принципы работы с ней будут очень похожи.

PEP 249 оперирует такими понятиями, как подключения и курсоры:

- Подключение — объект, в котором чаще всего указывается либо путь к файлу, либо путь к серверу. Он отвечает только за подключение к БД и, соответственно, отключение от нее
- Курсор — объект, в котором непосредственно производится работа с БД

In [1]:
# Установка библиотек для работы с БД
# обязательно запускайте данную ячейку перед началом новой рабочей сессии
%pip install psycopg[binary]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of psycopg[binary] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.5/179.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.2 MB/s eta 0:00:00


Далее приведем пример кода для подключения к базе данных. Основные моменты:
- Создание соединения
- Обработка отказа
- Создание курсора
- Выполнение запроса
- Обработка результатов

films https://disk.yandex.ru/d/niw2CgBJY4ojgA
countries https://disk.yandex.ru/d/ykdHwtz_fArYOQ

In [2]:
import psycopg

In [1]:
from psycopg import connect  # Импорт библиотек для работы с БД PostgreSQL (PostgreSQL является SQL совместимой БД)
from psycopg import OperationalError  # Обработчик ошибок
import pandas as pd
import numpy as np

try:  # Пытаемся совершить подключение к БД, с помощью параметров, выданных администратором сервера
    connection = psycopg.connect(
        user='da_student',
        password='knowledge_is_power_7823',
        host='ep-floral-union-a241upmf.eu-central-1.aws.neon.tech',
        dbname='countries'
    )
    connection.autocommit = True
    cur = connection.cursor()  # Создаем курсор управления

    """Создаем и выполняем запрос с помощью языка SQL"""

    cur.execute("SELECT country, land_area_km2, life_expectancy, official_language FROM countries_table WHERE \
    official_language = 'Russian' AND land_area_km2 > 600000")

    result = cur.fetchall() # получаем результат

    # Поместим результаты в DataFrame
    df = pd.DataFrame(result)
    df.columns = [row[0] for row in cur.description]
    print(df.head(7))
    connection.close()  # Не забываем закрыть соединение, иначе мы или другой пользователь может не попасть в следующую сессию

except OperationalError as error:  # В случае отказа - выводим предполагаемую причину
    print(f'Ошибка подключения к БД: {error}')


ModuleNotFoundError: No module named 'psycopg'

Как можно заметить, результат запроса — это список кортежей.

Метод .fetchall() возвращает все полученные элементы. Существует еще метод .fetchone(), возвращающий, как несложно догадаться, только первый элемент, и метод .fetchmany(n), возвращающий n первых записей.

Для запросов очень часто необходимо указывать какие-либо параметры, в нашем случае: год выпуска, продолжительность фильма и т. д. Для этого существует удобный синтаксис. Вместо значения в запросе указывается знак процента и s (%s), а затем вторым параметром в итерируемом объекте (чаще всего в кортеже) указываются необходимые значения для подстановки.

```SQL
result = cur.execute("""SELECT * FROM films
            WHERE year = %s and duration > %s""", (2010, 90)).fetchall()
```

**Важно не забыть, что, если мы указываем кортеж из одного элемента, нам все равно необходимо после него поставить запятую.**

```SQL
result = cur.execute("""SELECT * FROM Films
            WHERE year = %s""", (2009,)).fetchall()
```

## 3. Основы SQL

### 3.1 films_db.sqlite
Итак у нас есть база данных films_db. Давайте сначала посмотрим, как она выглядит в виде визуализации:

<img src="https://yastatic.net/s3/lyceum/content/images/second-year/pyqt-6/qt-6-1.png" alt="drawing" width="200"/>

Такие визуализации используют достаточно часто для наглядного представления таблиц и связей, и называются они ER-диаграммами (сокращение от Entity-Relation или Сущность-Отношение).

Итак, разберем, что есть в нашей базе данных. Есть две таблицы-сущности: **films** для хранения информации о фильмах и **genres** для хранения информации о жанрах. Таблица **genres** состоит всего из двух полей: **id и title**, в которых для каждого жанра хранится его идентификатор и название соответственно.

Таблица films чуть сложнее. Там есть поля **id, title, year, genre, duration** для хранения идентификатора фильма, его названия, года выпуска, идентификатора жанра и длительности в минутах.

Кроме того, как видно на диаграмме, между таблицами есть связь, которая говорит о том, что номер жанра **genre** у записи **films** соответствует записи в таблице **genres** с таким же значением идентификатора.

Для работы с базами данных был придуман специальный язык — **SQL** (structured query language — «язык структурированных запросов»).

### 3.2 Язык SQL
Основной командой для получения какой-либо информации из БД является команда SELECT. Ее базовый синтаксис выглядит так:

```SQL
SELECT перечень_полей FROM имя_таблицы
        WHERE условие
```

Кроме этого, есть и различные модификаторы этой команды. Например, ORDER BY ПОЛЕ — тогда результаты будут выведены в отсортированном виде по заданному полю или нескольким полям, а в условии может быть вложенный запрос.

Напишем наш первый запрос. Получим все фильмы, выпущенные в 2010 году.
```SQL
SELECT * FROM Films
    WHERE year = 2010
```
<img src="https://yastatic.net/s3/lyceum/content/images/second-year/pyqt-6/qt-6-5.png" alt="drawing" width="250"/>

### Пример запроса к базе данных с фильмами

In [ ]:
from psycopg import connect
from psycopg import OperationalError
import pandas as pd
import numpy as np

try:
    connection = psycopg.connect(
        user='da_student',
        password='knowledge_is_power_7823',
        host='ep-floral-union-a241upmf.eu-central-1.aws.neon.tech',
        dbname='films_database' # меняем название базы данных с countries на нужное
    )
    connection.autocommit = True
    cur = connection.cursor()  # Создаем курсор управления

    """Создаем и выполняем запрос с помощью языка SQL"""

    cur.execute("SELECT * FROM Films WHERE duration < 40")

    result = cur.fetchall() # получаем результат

    # Поместим результаты в DataFrame
    df = pd.DataFrame(result)
    df.columns = [row[0] for row in cur.description]
    print(df.head(7))
    connection.close()  # Не забываем закрыть соединение, иначе мы или другой пользователь может не попасть в следующую сессию

except OperationalError as error:  # В случае отказа - выводим предполагаемую причину
    print(f'Ошибка подключения к БД: {error}')

    id                         title  year  genre  duration
0    9          А к нам цирк приехал  1978      2        23
1   13          А потом оглянулся...  1980      2        31
2  243  Али-баба и сорок разбойников  1959     16        28
3  370                         Ангел  2007      9         6
4  409               Андалузский пес  1928      2        33
5  430                 Анна Каренина  2007      2        29
6  459                       Антракт  1924      2        14


### 3.3 Задание 1
- Подключитесь к базе данных: films_db
- Получите все фильмы, выпущенные в 2010 году
- Поместите результат в DataFrame (pandas)
- Не забудьте назначить соответвующие названия колонок (DF colums -> column_names SQL)
- Выведите первые 10 фильмов из DataFrame

In [ ]:
from psycopg import connect
from psycopg import OperationalError
import pandas as pd
import numpy as np

# Ваш код пишем тут

"""
Используйте пример выше
"""

'\nИспользуйте пример из 1-го раздела\n'

### 3.4 Задание 2

Условий может быть и несколько: работают все знакомые нам логические операторы NOT, AND и OR. Например, выберем фильмы, выпущенные после 2005 года с продолжительностью от 40 минут до 1,5 часов:
```SQL
SELECT * FROM Films
    WHERE year > 2005 AND duration >= 45 AND duration <= 90
```

<img src="https://yastatic.net/s3/lyceum/content/images/second-year/pyqt-6/qt-6-6.png" alt="drawing" width="700"/>


#### **Практика**
- Подключитесь к базе данных: films_db
- Получите все фильмы, выпущенные после 2005 года с продолжительностью от 40 минут до 1,5 часов
- Поместите результат в DataFrame (pandas)
- Не забудьте назначить соответвующие названия колонок (DF colums -> column_names SQL)
- Выведите первые 20 фильмов из DataFrame

In [ ]:
# Ваш код пишем тут


"""
Используйте код из 1-го задания
"""

'\nИспользуйте код из 1-го задания\n'

### 3.5 Задание 3 - Запросы по 2-м таблицам

А как вывести все фильмы определенного жанра, например, фантастика? Конечно, можно сходить в таблицу genres и посмотреть, какой id у жанра фантастика, а потом написать запрос вроде такого:

```SQL
SELECT title FROM films
    WHERE genre = 8
```

Но это плохой путь, потому что через некоторое время данные могут измениться, и id у фантастики может стать другим, тогда наш запрос будет давать ошибочный результат. Поэтому правильным решением в данной ситуации будет написать подзапрос, который сам найдет нам необходимое значение id.

```SQL
SELECT title FROM Films
    WHERE genre=(
SELECT id FROM genres
    WHERE title = 'фантастика')
```
Сначала выполнится внутренний запрос: из таблицы genres будет получен id для записи с title«Фантастика», а затем будет выполнено сравнение и выведен результат.

<img src="https://yastatic.net/s3/lyceum/content/images/second-year/pyqt-6/qt-6-1.png" alt="drawing" width="200"/>


#### **Практика**
- Подключитесь к базе данных: films_db
- Получите все фильмы в жанре "комедия" (если таковых не окажется - выберите жанр по вашему усмотрению, предварительно выведите все жанры из таблицы genres), выпущенные после 2007 года
- Поместите результат в DataFrame (pandas)
- Не забудьте назначить соответвующие названия колонок (DF colums -> column_names SQL)
- Выведите последние 5 фильмов из DataFrame

In [ ]:
# Ваш код

### 3.6 Задание 4 - Фильтрация

Помимо того, может быть выполнено сравнение не с одним элементом, а проверка на попадание в список. Это делается с помощью уже знакомого нам оператора IN. Например, так можно выбрать фильмы, продолжительность которых строго 45 или 90 минут:

```SQL
SELECT title, duration FROM Films
WHERE duration IN (45, 90)
```

Кроме уже операторов знакомых нам по Python, SQL содержит еще и ряд тех, которых в Python нет. Давайте рассмотрим несколько из них.

Оператор BETWEEN — проверяет, попадает ли заданное значение в диапазон (включая границы).

```SQL
SELECT * FROM Films
    WHERE (year > 2005) AND duration BETWEEN 45 AND 60
```

Оператор LIKE позволяет проверить, насколько похожа та или иная строка на заданный шаблон. Для шаблонов используются специальные символы:

% — обозначает любое количество, в том числе нулевое, любых символов
_ — обозначает один любой символ
Давайте получим список фильмов, у которых первая буква в названии — А и третья — к.
```SQL
SELECT * FROM Films
    WHERE title like 'А_к%'
```

<img src="https://yastatic.net/s3/lyceum/content/images/second-year/pyqt-6/qt-6-7.png" alt="drawing" width="400"/>

Оператор LIKE работает также в паре с NOT. Например, получим список фильмов, у которых третья буква в названии не равна д, а последняя не равна a.

```SQL
SELECT * FROM films
    WHERE title NOT LIKE '__д%а'
```
Кроме этого, есть возможность избавиться от повторов, используя в запросе специальный оператор — DISTINCT. Например, вот так можно получить список годов, в которые выходили фильмы в нашей базе данных, без повторений.
```SQL
SELECT DISTINCT year FROM Films
```

#### **Практика**
- Подключитесь к базе данных: films_db
- Напишите программу, которая выполняет запрос для получения списка фильмов, в названии которых упоминается Астерикс, но не упоминается Обеликс.
- Вывести необходимо все названия, каждое с новой строки.

In [ ]:
# Ваш код




### 3.7 Задание 5 - Статистика

#### **Практика**
- Подключитесь с базе данных: films_db
- Напишите программу, которая находит все фильмы продолжительностью от 30 до 120 минут. Формат выборки полей films.title, genre.title (Название фильма, название жанра)
- Поместите выборку в DataFrame
- Выведите основные статистические показатели
- Выведите фильм с кратчайшим названием
- Посчитайте количество добрых фильмов (содержащих корень "добр" в названии)
- Посчитайте количество злых фильмов
- Посчитайте отношение добрых фильмов к злым


In [ ]:
# Ваш код тут

## 4. Создание таблицы и занесение данных.

Для создания таблицы используется конструкция:
```SQL
CREATE TABLE humans (
    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100),
    age SMALLINT);
```
Где:
- id - первичный ключ - число - автоматически инкрементируемое
- name - строковая переменная - размером 100 символов
- age - целочисленная переменная - малого размера

Справка по типам данных MySQL: https://dev.mysql.com/doc/refman/8.0/en/data-types.html

Для занесения данных используется конструкция:
```SQL
INSERT INTO humans (name, age)
    VALUES('Петя', 24), ('Таня', 22);
```
Для сохранения внесенных данных используется отдельный запрос:
```SQL
COMMIT;
```
Для проверки внесенных данных воспользуйтесь конструкцией SELECT после закрытия соединения и поторного подключения - чтобы убедиться в выполнении транзакции:
```SQL
SELECT * FROM humans;
```

In [3]:
from psycopg import connect
from psycopg import OperationalError
import pandas as pd
import numpy as np

try:
    connection = psycopg.connect(
        user='da_student',
        password='knowledge_is_power_7823',
        host='ep-floral-union-a241upmf.eu-central-1.aws.neon.tech',
        dbname='films_database'
    )
    connection.autocommit = True
except OperationalError as error:  # В случае отказа - выводим предполагаемую причину
    print(f'Ошибка подключения к БД: {error}')

cur = connection.cursor()  # Создаем курсор управления

# снимите комментарии ниже для создания таблицы в базе данных
#cur.execute("""CREATE TABLE film_watch (
#    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
#    film_id VARCHAR(100))""")


cur.execute("""INSERT INTO film_watch (film_id)
    VALUES(100);""")

cur.execute("""COMMIT;""")


cur.execute("""SELECT * FROM film_watch;""")

result = cur.fetchall() # получаем результат
print(result)

connection.close()  # Не забываем закрыть соединение, иначе мы или другой пользователь может не попасть в следующую сессию


UndefinedTable: relation "film_watch" does not exist
LINE 1: SELECT * FROM film_watch;
                      ^

Запустите пример запросов к БД в цикле - для имитации поисковой системы.

In [16]:
from psycopg import connect
from psycopg import OperationalError
import pandas as pd

try:
    connection = psycopg.connect(
        user='da_student',
        password='knowledge_is_power_7823',
        host='ep-floral-union-a241upmf.eu-central-1.aws.neon.tech',
        dbname='films_database'
    )
    connection.autocommit = True
except OperationalError as error:  # В случае отказа - выводим предполагаемую причину
    print(f'Ошибка подключения к БД: {error}')

cur = connection.cursor()  # Создаем курсор управления

query = input("Введите название или часть названия искомого фильма: ")
while query != '':
  query = '%' + query + "%"
  cur.execute("""SELECT title, year, duration FROM films WHERE title LIKE %s;""", (query,))
  result = cur.fetchall() # получаем результат
  df = pd.DataFrame(result)
  print(df)
  df.columns = [row[0] for row in cur.description]
  print(df)
  print('\n', 'для выхода нажмите Enter', '\n')
  query = input("Введите название или часть названия искомого фильма: ")

connection.close()  # Не забываем закрыть соединение, иначе мы или другой пользователь может не попасть в следующую сессию


Введите название или часть названия искомого фильма: опять
                                                   0     1    2
0                      А в России опять окаянные дни  1990  133
1  На Дерибасовской хорошая погода или на Брайтон...  1992   90
2                                     Оно опять живо  1978   91
3              Полицейская академия III: опять учеба  1986   90
                                               title  year  duration
0                      А в России опять окаянные дни  1990       133
1  На Дерибасовской хорошая погода или на Брайтон...  1992        90
2                                     Оно опять живо  1978        91
3              Полицейская академия III: опять учеба  1986        90

 для выхода нажмите Enter 

Введите название или часть названия искомого фильма: 


### 4.1 Сохранение истории запросов
Дополните код, представленный выше, для сохранения истории запросов к БД. Программа должна обеспечивать:


*   Хранение текста запросов
*   Хранение количества выданных результатов
*   Первую полную строку выдачи результата

Хранение этой структуры вы должны организовать в файле *.CSV

Пример хранящихся данных:


---


Запрос  Кол-во ответов:    1-я строка выдачи:

собак   28                Автомобиль, скрипка и собака Клякса  1974  101

термин  9                 Женщина-терминатор                   1992  90



---





In [ ]:
# Разместите решение тут

### 4.2 Сохрание просмотренных фильмов в БД
Используя код поиска фильмов и код добавления записей в БД (*Раздел №4*), создайте программу позволяющую:

*   Искать фильм по названию
*   Выбрать запись из поисковой выборки с помощью индекса
*   Сохранить номер фильма в таблицу film_watch
*   Программа должна работать непрерывно, обеспечивая цикл работы: поиск - выбор - сохранение - поиск ....
*   После окончания цикла (выход из цикла - нажатие Enter - пустая строка) выведите из БД все просмотренные фильмы из таблицы


``` SQL
SELECT title FROM films, film_watch  WHERE films.id == film_watch.film_id
```


*   После вывода удалите дубликаты записей с помощью:
ALTER IGNORE TABLE \`table name\` ADD UNIQUE INDEX(index);
*   Выведите просмотренные фильмы повторно

### Мы работаем с базой world
https://prowebmastering.ru/sample-mysql-base.html

## SQL Style
https://www.sqlstyle.guide/ru/